# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп»: разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и обосновать корректность его работы. Защитить данные нужно так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

**P.S.** *Подбирать наилучшую модель не требуется.*

### План работы: 

1. Загрузить и изучить данные.

2. Ответьте на вопрос и обоснуйте решение:
```
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

- Изменится. Приведите примеры матриц.
- Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
```

3. Предложить алгоритм преобразования данных для решения задачи с обоснованием, почему качество линейной регрессии не поменяется.

4. Запрограммировать этот алгоритм, применив матричные операции. 
    + Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования (применяя метрику R2).

## Загрузка данных

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")


import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import plotly.express as px

from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline


RANDOM_STATE=12345

In [6]:
pth1 = '/datasets/'
pth2 = 'C:/Users/Солнышко/Documents/'
pth3 = 'C:/Users/Home/Documents/Яндекс/'

if os.path.exists(pth1):
    data = pd.read_csv(pth1 +'insurance.csv')
elif os.path.exists(pth2):
    data = pd.read_csv(pth2 +'insurance.csv')
elif os.path.exists(pth3):
    data = pd.read_csv(pth3 +'insurance.csv')
else:
    print('Something is wrong')
    
    
# сбор данных о датафрейме:
def data_info(data):
    print(f'''
    ----------------------------------------
    Первые строки датафрейма:
    ----------------------------------------''')
    display(data.head())
    print(f'''
    ----------------------------------------
    Последние строки датафрейма:
    ----------------------------------------''')
    display(data.tail())
    print(f'''
    ----------------------------------------
    Общая информация:
    ----------------------------------------''')
    print(data.info())
    print(f'''
    ----------------------------------------
    Дупликаты:
    ----------------------------------------''')
    print(data.duplicated().sum())
    print(f'''
    ----------------------------------------
    Пропуски:
    ----------------------------------------''')
    display(round(data.isna().sum(),))

In [7]:
data_info(data)


    ----------------------------------------
    Первые строки датафрейма:
    ----------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0



    ----------------------------------------
    Последние строки датафрейма:
    ----------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0



    ----------------------------------------
    Общая информация:
    ----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None

    ----------------------------------------
    Дупликаты:
    ----------------------------------------
153

    ----------------------------------------
    Пропуски:
    ----------------------------------------


Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

### Данные:

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

Датасет небольшой - всего 5000 строк. Пропусков нет, а вот дупликаты есть, однако нельзя только сказать, это дупликаты в данных или действительно совпало, что у нескольких людей одинаковые характеристики. Однако, все таки удалим дупликаты

In [8]:
data = data.drop_duplicates()
data.duplicated().sum()

0

Также я бы изменила колонку с возрастом - он не может быть дробный, можно поставить int.

In [9]:
data['Возраст'] = data['Возраст'].astype('int64')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   int64  
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 227.2 KB


Далее, можем разделять данные:

In [13]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.25, random_state=RANDOM_STATE)

In [14]:
print(features_train.shape, features_valid.shape, target_train.shape, target_valid.shape)

(3635, 4) (1212, 4) (3635,) (1212,)


Краткий вывод по проделанной работе:

- загрузили данные
- изучили форматы, преобразовали то, что нужно
- проверили на пропуски и дупликаты, убрали вторую категорию
- разделили данные на выборки

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

**Ответ:** не изменится

**Обоснование:**

Возпользуемся свойствами матриц:

$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$

Определим полную формулу предсказаний для исходных данных:

$$
a = X(X^T X)^{-1} X^T y
$$

***Для обработанных данных:***

Обозначим обратимую матрицу как $Р$ размерностью $(nxn)$. Умножим $X$ на $Р$:

$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$

$$
a = XP((XP)^T XP)^{-1} (XP)^T y
$$

Тогда:

$$
a = XP(P^T X^T XP)^{-1} P^T X^T y
$$

$$
a = XP (X^T X P)^{-1} (P^T)^{-1} P^T X^T y
$$

$$
a = XP (P)^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

$$
a = XE (X^T X)^{-1} E X^T y
$$

$$
a = X(X^T X)^{-1} X^T y
$$

То есть при изменении матриц, результат останется тем же, потому что изменение признаков в данном случае не изменяет алгоритм, по которому высчитываются метрики.

Размерности различных матриц:

$X = (mxn)$

$X^T = (nxm)$

$X^TX = (nxm)*(mxn) = (nxn)$

$P = (nxn)$

$P^T = (nxn)$

$XP = (mxn)*(nxn) = (mxn)$

$(XP)^T = ((mxn)*(nxn))^T = (mxn)^T = (nxm)$

$P^T X^T = (nxn)*(nxm) = (nxm)$

## Алгоритм преобразования

**Алгоритм и его обоснование**

Как мы выяснили в прошлом этапе, от домножения на обратимую матрицу, результат не поменяется - модель будет работать все так же четко. Это и можно использовать для шифрования исходных данных - это не повлияет на качество работы модели, но будет гораздо меньше шансов найти исходные признаки - для этого нужно будет знать матрицу, на которую мы домножали.

## Проверка алгоритма

Хочу проверить и наш алгоритм, который мы писали в последнем задании, и модель sklearn - в теории, они должны показать одинаковые результаты, и если так и будет, то для простоты написания будем использовать модель sklearn.

In [15]:
class LinearRegressor:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T@X)@X.T@y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Проверим для начала работу моделей на *сырых* данных

In [16]:
model = LinearRegressor()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print('r2:', r2_score(target_valid, predictions))
print('mse:', mean_squared_error(target_valid, predictions))

r2: 0.4230772749214825
mse: 0.11955009374099718


In [18]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print('r2:', r2_score(target_valid, predictions))
print('mse:', mean_squared_error(target_valid, predictions))

r2: 0.42307727492147584
mse: 0.11955009374099854


Разница незначительна $->$ используем. Необходимо создать обратимую матрицу: (P.S.: *квадратная матрица обратима, когда ее определитель не равен нулю*)

In [22]:
invert_matr = np.random.normal(size=(features_train.shape[1], features_train.shape[1]))
print(np.linalg.det(invert_matr))
print(np.linalg.det(invert_matr)==0)
features_train = features_train @ invert_matr
features_valid = features_valid @ invert_matr

0.8114105172664603
False


Определитель нулю не равен, следовательно, у матрицы, которую мы используем для шифрования данных есть обратная, а мы можем идти дальше :)

In [23]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print('r2:', r2_score(target_valid, predictions))
print('mse:', mean_squared_error(target_valid, predictions))

r2: 0.42307727497001457
mse: 0.11955009373094035


Как мы видим, результаты даже немного улучшились, но эта разница заметна только в 13 знаке после запятой

In [24]:
features_train.head()

,0,1,2,3
4599,53458.827372,-107148.452821,29015.396082,-133840.255006
3882,49728.805459,-99684.309215,27015.915579,-124437.310388
4705,27465.558677,-55054.253153,14927.567411,-68706.730071
1400,38057.069731,-76294.585973,20714.181961,-95120.868078
728,31879.980919,-63908.955585,17352.572479,-79678.367428


Однако смотря на таблицы с признаками, тяжело понять, где что - и без знания invert_matr - тяжело будет привести данные в изначальный вид.

### Итоговый вывод.

В ходе работы как таковой обработки признаков не было, однако нам удалось понять и доказать, почему при домножении на обратимуюю матрицу результат работы алгоритма и метрики не поменяются, и как нам это может помочь в шифровании данных.

Был предложен алгоритм шифрования в виде "домножения на обратимую матрицу", так как изменяет данные оно прилично, однако не зная самой матрицы - изначальные данные практически невозможно восстановить.